In [30]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
from scipy.special import factorial2

hbarc = 197.3 #MeV fm
hbar = 6.582E-22 #MeV s
alpha = 1/137 
mP = 938.272 # Mev/c^2
#muN = 3.152E-14 # MeV / T
muN2 =  alpha*hbarc**3*2*mP**-2 # nuclear magneton squared

def Thalf_width(E): #converts energy width (MeV) to Thalf in seconds
    return hbar * np.log(2)/E

def Thalf_width_kev(E):
    if E==0.:
        return None
    return Thalf_width(E/1000)

def times_so_far(ls):
    out = [0]*len(ls)
    for i in range(len(ls)):
        out[i] = ls[:i+1].count(ls[i])
    return out

def j_to_twoj(x):
    if '/2' in x:
        return int(x.strip('/2'))
    else:
        return 2*int(x.strip('()'))
    
def thalf_units_to_sec(units,value):
    m=0
    if units=='PS':
        m = 10**-12
    elif units=='NS':
        m = 10**-9
    elif units=='FS':
        m = 10**-15
    elif units=='US':
        m = 10**-6
    elif units=='AS':
        m = 10**-18
    elif units=='KEV':
        pass
    if m==0:
        return Thalf_width_kev(value)
    else:
        return m*value
    
def my_float(x):
    try:
        return float(x)
    except:
        return 0.0
    
def fLambda(l):
    return 8*np.pi*(l+1)/(l*(factorial2(2*l+1)**2))

# def BEl(l,Egamma,Thalf,I,units):
#     if units == 1: # [B] = fm^2l
#         B = (I/100)*(np.log(2)/Thalf)*fLambda(l)**-1*hbar*hbarc**(2*l+1)*(Egamma**(-2*l-1))
#     elif units == 2: # [B] = e^2 fm^2l
#         B = (I/100)*(np.log(2)/Thalf)*fLambda(l)**-1*hbar*hbarc**(2*l+1)*(Egamma**(-2*l-1))
#         B = B/(alpha*hbarc)
#     elif units == 3: # [B] = e^2 b^l
#         B = (I/100)*(np.log(2)/Thalf)*fLambda(l)**-1*hbar*hbarc**(2*l+1)*(Egamma**(-2*l-1))
#         B = 10**(-2*l)*B/(alpha*hbarc)
#     return B

def BM1(Egamma,Thalf):
    return (np.log(2)/Thalf) * (1/1.76E13) * Egamma**-3 

# def BE2W(A,Egamma,Thalf,I):
#     lamW = 7.3E7*A**(4/3)*Egamma**5;
#     lamExp = (np.log(2)/Thalf)*(I/100);
#     return lamExp/lamW

# def Bweisskopf(l,A):
#     # Weisskopf (single-particle) estimate in e^2 fm^2l
#     return (1/(4*np.pi)) * (3/(3+l))**2 * (1.2*A**(1/3))**(2*l)

# def BEl_in_Wu(l,Egamma,Thalf,I,A):
#     return BEl(l,Egamma,Thalf,I,2) / Bweisskopf(l,A)

# def BWu_convert(BEl_Wu,l,A):
#     return BEl_Wu*Bweisskopf(l,A)

BM1_Weisskopf = (10/np.pi)*0.75**2


In [34]:
# df = pd.read_csv('sd_E2_data.csv',sep=r'\s*,\s*',header=0, encoding='ascii', engine='python')
df = pd.read_csv('M1_ensdf.csv')
df = df.rename(columns=lambda x: x.strip())

In [35]:
df

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.)
0,16,O \t,8,8,9844.50,5,2+ \t,0.62 KEV 10 \t,2927.10,8 \t,[M1] \t,0.00420,\t8
1,18,O \t,8,10,3920.44,14,2+ \t,18.4 FS 20 \t,1938.00,\t,M1 \t,0.14000,\t2
2,18,O \t,8,10,5254.80,9,2+ \t,7.0 FS 3 \t,1334.00,\t,M1 \t,0.11100,\t8
3,18,O \t,8,10,7116.90,12,4+ \t,17 FS LT \t,3562.00,\t,M1 \t,0.07100,\t16
4,18,O \t,8,10,8213.00,4,2+ \t,1.0 KEV 8 \t,4293.00,\t,M1 \t,0.00720,\t30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,39,K \t,19,20,4930.30,4,3/2+ \t,21 FS 9 \t,2407.40,3 \t,M1+E2 \t,0.01300,\t<
570,39,K \t,19,20,4930.30,4,3/2+ \t,21 FS 9 \t,4931.40,12 \t,M1+E2 \t,0.00250,\t+58-15
571,39,K \t,19,20,7776.31,22,17/2+ \t,1.0 PS 3 \t,1301.18,11 \t,M1+E2 \t,0.00250,\t+26-12
572,39,CA \t,20,19,4020.70,17,1/2+ \t,0.42 PS 12 \t,1553.80,\t,[M1] \t,0.01100,\t+7-4


In [36]:
df[df['A']==22]

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.)
83,22,F \t,9,13,709.10,3,(2+) \t,57 FS 28 \t,637.40,2 \t,[M1] \t,1.500000,\t8
100,22,NE \t,10,12,4456.20,9,2+ \t,3.7 FS 25 \t,3181.40,\t,M1+E2 \t,0.180000,\t12
101,22,NE \t,10,12,5329.60,13,1+ \t,1.2 FS 3 \t,4054.60,\t,M1+E2 \t,0.020000,\t11
102,22,NE \t,10,12,5363.40,11,2+ \t,69 FS 12 \t,4088.40,\t,M1+E2 \t,0.003900,\t8
103,22,NE \t,10,12,5523.30,6,(4)+ \t,21 FS 3 \t,2166.10,5 \t,M1 \t,0.102000,\t15
104,22,NE \t,10,12,5641.20,7,3+ \t,3 FS LT \t,4366.10,10 \t,M1+E2 \t,0.059000,\t>
105,22,NE \t,10,12,6119.90,16,2+ \t,14 FS 7 \t,4844.80,\t,M1+E2 \t,0.001700,\t10
106,22,NE \t,10,12,6119.90,16,2+ \t,14 FS 7 \t,1663.60,\t,M1+E2 \t,0.012000,\t8
107,22,NE \t,10,12,6345.10,10,4+ \t,13 FS 3 \t,2987.70,9 \t,M1+E2 \t,0.043000,\t12
108,22,NE \t,10,12,6819.40,16,2+ \t,3 FS LT \t,2363.10,\t,M1+E2 \t,0.021000,\t>


In [37]:
def isNaN(num):
    return num != num

for i in df.index:
    if isNaN(df['A'].loc[i]):
#         print(df['A'].loc[i])
        df = df.drop(i)
    
df['A'] = df['A'].map(int)
df['Z'] = df['Z'].map(int)
df['N'] = df['N'].map(int)

In [38]:
# df['Element'] = df['Element'].apply(lambda x: str.strip(x))

for key in df:
    try:
        df[key] = df[key].apply(lambda x: str.strip(x))
    except:
        pass


df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Element'] = df['Element'].apply(str.capitalize)

In [39]:
df = df.rename(columns={'B(W.u.)':'B_exp (W.u.)'})
df = df.rename(columns={'B Unc (W.u.)':'B_exp Unc (W.u.)'})

In [40]:
# df['Jpi'] = df['Jpi'].map(lambda x: str(x).strip().strip('()+'))
# df = df.rename(columns={'Jpi':'Ji'})
df['Ji'] = df['Ji'].map(lambda x: str(x).strip().strip('()+'))
# df = df.rename(columns={'Jpi':'Ji'})

df['twoJi'] = df['Ji'].map(j_to_twoj)

In [41]:
for i in df.index:
    string = df['B_exp Unc (W.u.)'].loc[i]
    try:
        up,down = string.strip('+').split('-')
        df['B_exp Unc (W.u.)'].loc[i] = max(float(up),float(down))
    except:
        pass
    if string=='':
        df['B_exp Unc (W.u.)'].loc[i] = 0.

/tmp/ipykernel_2553/4196107385.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['B_exp Unc (W.u.)'].loc[i] = 0.
/tmp/ipykernel_2553/4196107385.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['B_exp Unc (W.u.)'].loc[i] = max(float(up),float(down))


In [42]:
df

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.),twoJi
0,16,O,8,8,9844.50,5,2,0.62 KEV 10,2927.10,8,[M1],0.00420,8,4
1,18,O,8,10,3920.44,14,2,18.4 FS 20,1938.00,,M1,0.14000,2,4
2,18,O,8,10,5254.80,9,2,7.0 FS 3,1334.00,,M1,0.11100,8,4
3,18,O,8,10,7116.90,12,4,17 FS LT,3562.00,,M1,0.07100,16,8
4,18,O,8,10,8213.00,4,2,1.0 KEV 8,4293.00,,M1,0.00720,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,39,K,19,20,4930.30,4,3/2,21 FS 9,2407.40,3,M1+E2,0.01300,<,3
570,39,K,19,20,4930.30,4,3/2,21 FS 9,4931.40,12,M1+E2,0.00250,58.0,3
571,39,K,19,20,7776.31,22,17/2,1.0 PS 3,1301.18,11,M1+E2,0.00250,26.0,17
572,39,Ca,20,19,4020.70,17,1/2,0.42 PS 12,1553.80,,[M1],0.01100,7.0,1


In [10]:
# df[['Thalf Value','Thalf Units','Thalf Unc']] = df['Thalf'].str.split(expand=True)

# df = df[df['Thalf Unc']!='LT']
# df = df[df['Thalf Unc']!='LE']
# df = df[df['Thalf Unc']!='GT']
# # df = df[df['I_gamma Unc']!='AP']

# df['Thalf Unc'] = df['Thalf Unc'].fillna(0.0)
# df['E_gamma Unc (keV)'] = df['E_gamma Unc (keV)'].fillna(0.0)
# # df['I_gamma Unc'] = df['I_gamma Unc'].fillna(0.0)

# for i in df.index:
#     string = df['Thalf Unc'].loc[i]
#     try:
#         up,down = string.strip('+').split('-')
#         df['Thalf Unc'].loc[i] = max(float(up),float(down))
#     except:
#         pass

# df['Thalf (sec)'] = df[['Thalf Units','Thalf Value']].apply(lambda row: thalf_units_to_sec(row['Thalf Units'],float(row['Thalf Value'])),axis=1,result_type='expand')
# df['Thalf Unc (sec)'] = df[['Thalf Units','Thalf Unc']].apply(lambda row: thalf_units_to_sec(row['Thalf Units'],float(row['Thalf Unc'])),axis=1,result_type='expand')

In [11]:
# for col in ['I_gamma','I_gamma Unc','E_gamma (keV)','E_gamma Unc (keV)']:
#     df[col] = df[col].apply(my_float)

# df['B_exp (e^2fm^4)'] = df[['E_gamma (keV)','Thalf (sec)','I_gamma']].apply(lambda row: BEl(2,row['E_gamma (keV)']/1000,row['Thalf (sec)'],row['I_gamma'],2),axis=1,result_type='expand')

# NOTE: the 5 on the E_gamma term below comes from the derivative dB/dE and is 5 = |-2*lambda - 1|
# df['B_exp Unc (e^2fm^4)'] = df.apply(lambda row: row['B_exp (e^2fm^4)']*np.sqrt( (row['I_gamma Unc']/row['I_gamma'])**2 + (5 * row['E_gamma Unc (keV)']/row['E_gamma (keV)'])**2 + (row['Thalf Unc (sec)']/row['Thalf (sec)'])**2 ) , axis=1 ,result_type='expand' ) 

In [12]:
# df['B_exp (mu_N^2)'] = -9.
# df['B_exp Unc (mu_N^2)'] = -9.

# df['B_exp Unc (W.u.)'] = df['B_exp Unc (W.u.)'][df['B_exp Unc (W.u.)'] != '<']
# df['B_exp Unc (W.u.)'] = df['B_exp Unc (W.u.)'][df['B_exp Unc (W.u.)'] != '>']

# for i in df.index:
# #     print(df['B_exp (W.u.)'].loc[i])
# #     print(df['B_exp Unc (W.u.)'].loc[i])
# #     print(df['A'].loc[i])
#     df['B_exp (mu_N^2)'].loc[i] = BM1_Weisskopf * float(df['B_exp (W.u.)'].loc[i])
#     df['B_exp Unc (mu_N^2)'].loc[i] = BM1_Weisskopf * float(df['B_exp Unc (W.u.)'].loc[i])
    

In [43]:
df['Nucleus'] = df.apply(lambda x: str(int(x['A']))+x['Element'], axis=1)

In [44]:
df_levels = pd.read_csv('sd_levels_20mev.csv')

df_levels['E(keV)'] = None
df_levels['dE(keV)'] = None
    
for i in df_levels.index:
#     print(df_levels['Elevel(keV)'].iloc[i])
    E = df_levels['Elevel(keV)'].loc[i].split()
    if len(E)==1:
        df_levels['E(keV)'].loc[i] = float(E[0].replace('?',''))
        df_levels['dE(keV)'].loc[i] = None
    else:
        df_levels['E(keV)'].loc[i] = float(E[0].replace('?',''))
        df_levels['dE(keV)'].loc[i] = float(E[1].replace('?',''))  

# df_levels = df_levels.drop('unclean_E',axis=1)
df_levels = df_levels.drop('T1/2',axis=1)
# df_levels = df_levels.drop('unclean_Jpi',axis=1)
# df_levels = df_levels.drop('unclean_T1/2',axis=1)
df_levels['Include'] = True
for char in [',','&','-','GE',':','TO']:
    df_levels['Include'] *= df_levels['Jpi'].map(lambda x : not char in str(x))
df_levels = df_levels.drop(df_levels[df_levels['Include']==False].index)
for char in ['+','(',')','[',']']:
    df_levels['Jpi'] = df_levels['Jpi'].map(lambda x: str(x).replace(char,''))
df_levels = df_levels.rename(columns={'Jpi':'J'})
df_levels = df_levels.drop(df_levels[df_levels['J']==''].index)
df_levels['twoJ'] = df_levels['J'].map(j_to_twoj)

df_levels['Element'] = df_levels['Nucleus'].map(lambda x : x[2:])
df_levels['Element'] = df_levels['Element'].apply(str.capitalize)
df_levels['A'] = df_levels['Nucleus'].map(lambda x : x[:2])
df_levels['Nucleus'] = df_levels.apply(lambda x: str(int(x['A']))+x['Element'], axis=1)

In [45]:
df_levels['Nucleus'].unique()

array(['11O', '12O', '14O', '15O', '16O', '17O', '18O', '19O', '20O',
       '21O', '22O', '23O', '24O', '26O', '15F', '16F', '17F', '18F',
       '19F', '20F', '21F', '22F', '23F', '24F', '25F', '26F', '27F',
       '16Ne', '18Ne', '19Ne', '20Ne', '21Ne', '22Ne', '23Ne', '24Ne',
       '25Ne', '26Ne', '27Ne', '28Ne', '29Ne', '30Ne', '32Ne', '34Ne',
       '17Na', '19Na', '20Na', '21Na', '22Na', '23Na', '24Na', '25Na',
       '26Na', '27Na', '28Na', '29Na', '30Na', '31Na', '33Na', '35Na',
       '20Mg', '21Mg', '22Mg', '23Mg', '24Mg', '25Mg', '26Mg', '27Mg',
       '28Mg', '29Mg', '30Mg', '31Mg', '32Mg', '34Mg', '36Mg', '38Mg',
       '40Mg', '21Al', '22Al', '23Al', '24Al', '25Al', '26Al', '27Al',
       '28Al', '29Al', '30Al', '31Al', '32Al', '33Al', '35Al', '39Al',
       '22Si', '23Si', '24Si', '25Si', '26Si', '27Si', '28Si', '29Si',
       '30Si', '31Si', '32Si', '33Si', '34Si', '35Si', '36Si', '37Si',
       '38Si', '39Si', '40Si', '42Si', '44Si', '25P', '26P', '27P', '28P',
     

In [46]:
df_levels['n'] = None
for nucleus in df_levels['Nucleus'].unique():
    ls = list(df_levels[df_levels['Nucleus']==nucleus]['twoJ'])
#     print(ls)
    tsf = times_so_far(ls)
    j = 0
    for i in df_levels[df_levels['Nucleus']==nucleus].index:
#         print(i)
        df_levels['n'].loc[i] = tsf[j]
        j+=1

/tmp/ipykernel_2553/602153835.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_levels['n'].loc[i] = tsf[j]


In [47]:
df_levels

,Nucleus,Elevel(keV),J,E(keV),dE(keV),Include,twoJ,Element,A,n
0,11O,490,5/2,490.0,None,True,5,O,11,1
1,11O,2120,5/2,2120.0,None,True,5,O,11,2
2,12O,0,0,0.0,None,True,0,O,12,1
3,12O,1.62E3 11,0,1620.0,11.0,True,0,O,12,2
4,12O,1968 52,2,1968.0,52.0,True,4,O,12,1
...,...,...,...,...,...,...,...,...,...,...
7876,52Ca,2563 1,2,2563.0,1.0,True,4,Ca,52,1
7877,54Ca,0,0,0.0,None,True,0,Ca,54,1
7878,54Ca,2043 19,2,2043.0,19.0,True,4,Ca,54,1
7879,56Ca,0,0,0.0,None,True,0,Ca,56,1


In [48]:
df['Jf'] = None
df['twoJf'] = None
df['ni'] = None
df['nf'] = None
df['Include'] = True

max_deltaJ = 1
tol_energy = 0.5
tol_gamma = 0.5

# for i in df.index:
for i in df.index:
    nuc_cond = (df_levels['Nucleus']==df['Nucleus'].loc[i])
    i_J_cond = (df_levels['twoJ']==df['twoJi'].loc[i])
    i_E_cond = (tol_energy > abs(df_levels['E(keV)'] - df['Energy (keV)'].loc[i]))
    f_J_cond = ( 2*max_deltaJ >= abs(df_levels['twoJ'] - df['twoJi'].loc[i]))
    f_E_cond = (tol_gamma > abs(df_levels['E(keV)'] - df['Energy (keV)'].loc[i] + df['E_gamma (keV)'].loc[i]))
    i_condition =  nuc_cond &  i_J_cond  &  i_E_cond
    f_condition =  nuc_cond &  f_J_cond  &  f_E_cond
    state_i = df_levels[i_condition]
    state_f = df_levels[f_condition]
    
#     print('len=',len(state_i))
    
    if (len(state_i)!=1)|(len(state_f)!=1):
        df['Include'].loc[i] = False
        print(f"-------------------------------------------")
        print(f"Excluding transition i = {i}")
        print(f'Matched # initial states: {len(state_i)}' )
        print(f'Matched # final   states: {len(state_f)}' )
        print(f"Matched # Ji             : { len( df_levels[ nuc_cond & i_J_cond ] ) }")
        print(f"Matched # Jf             : { len( df_levels[ nuc_cond & f_J_cond ] ) }")
        print(f"Matched # Ei             : { len( df_levels[ nuc_cond & i_E_cond ] ) }")
        print(f"Matched # Ef             : { len( df_levels[ nuc_cond & f_E_cond ] ) }")
        print(state_i)
        print(state_f)
        print(df.loc[i])

        continue
        
    df['Jf'].loc[i] = list(state_f['J'])[0]
    df['twoJf'].loc[i] = int(state_f['twoJ'])
    df['ni'].loc[i] = int(state_i['n'])
    df['nf'].loc[i] = int(state_f['n'])
    
#     print(state_i)
#     print(state_f)
#     print(df.loc[i])
    

/tmp/ipykernel_2553/929582128.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_2553/929582128.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['twoJf'].loc[i] = int(state_f['twoJ'])
/tmp/ipykernel_2553/929582128.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ni'].loc[i] = int(state_i['n'])
/tmp/ipykernel_2553/929582128.py:44: SettingWithCopyWarning: 
A value is

-------------------------------------------
Excluding transition i = 7
Matched # initial states: 1
Matched # final   states: 0
Matched # Ji             : 3
Matched # Jf             : 13
Matched # Ei             : 1
Matched # Ef             : 0
    Nucleus Elevel(keV)  J   E(keV) dE(keV)  Include  twoJ Element   A  n
248     18F   1041.55 8  0  1041.55     8.0     True     0       F  18  1
Empty DataFrame
Columns: [Nucleus, Elevel(keV), J, E(keV), dE(keV), Include, twoJ, Element, A, n]
Index: []
A                            18
Element                       F
Z                             9
N                             9
Energy (keV)            1041.55
Energy Unc (keV)              8
Ji                            0
Thalf                1.77 FS 31
E_gamma (keV)            1041.0
E_gamma Unc (keV)              
M Gamma                      M1
B_exp (W.u.)               10.9
B_exp Unc (W.u.)             21
twoJi                         0
Nucleus                     18F
Jf                  

/tmp/ipykernel_2553/929582128.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_2553/929582128.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_2553/929582128.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['twoJf'].loc[i] = int(state_f['twoJ'])
/tmp/ipykernel_2553/929582128.py:43: SettingWithCopyWarning: 
A value is trying

-------------------------------------------
Excluding transition i = 89
Matched # initial states: 1
Matched # final   states: 0
Matched # Ji             : 5
Matched # Jf             : 14
Matched # Ei             : 1
Matched # Ef             : 0
    Nucleus Elevel(keV)    J  E(keV) dE(keV)  Include  twoJ Element   A  n
674    21Ne    2866.6 2  9/2  2866.6     2.0     True     9      Ne  21  1
Empty DataFrame
Columns: [Nucleus, Elevel(keV), J, E(keV), dE(keV), Include, twoJ, Element, A, n]
Index: []
A                         21
Element                   Ne
Z                         10
N                         11
Energy (keV)          2866.6
Energy Unc (keV)           2
Ji                       9/2
Thalf                40 FS 4
E_gamma (keV)         1120.0
E_gamma Unc (keV)          3
M Gamma                M1+E2
B_exp (W.u.)            0.24
B_exp Unc (W.u.)           3
twoJi                      9
Nucleus                 21Ne
Jf                      None
twoJf                   None
ni  

/tmp/ipykernel_2553/929582128.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_2553/929582128.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_2553/929582128.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_2553/929582128.py:42: SettingWithCopyWarning: 
A value is trying to be set on

...

/tmp/ipykernel_2553/929582128.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_2553/929582128.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_2553/929582128.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['twoJf'].loc[i] = int(state_f['twoJ'])
/tmp/ipykernel_2553/929582128.py:43: SettingWithCopyWarning: 
A value is trying

-------------------------------------------
Excluding transition i = 408
Matched # initial states: 1
Matched # final   states: 0
Matched # Ji             : 5
Matched # Jf             : 9
Matched # Ei             : 1
Matched # Ef             : 0
     Nucleus Elevel(keV)  J  E(keV) dE(keV)  Include  twoJ Element   A  n
2696    30Si    7223.2 4  4  7223.2     4.0     True     8      Si  30  3
Empty DataFrame
Columns: [Nucleus, Elevel(keV), J, E(keV), dE(keV), Include, twoJ, Element, A, n]
Index: []
A                          30
Element                    Si
Z                          14
N                          16
Energy (keV)           7223.2
Energy Unc (keV)            4
Ji                          4
Thalf                14 FS LT
E_gamma (keV)          1943.0
E_gamma Unc (keV)          11
M Gamma                 M1+E2
B_exp (W.u.)            0.074
B_exp Unc (W.u.)            >
twoJi                       8
Nucleus                  30Si
Jf                       None
twoJf              

/tmp/ipykernel_2553/929582128.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_2553/929582128.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_2553/929582128.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['twoJf'].loc[i] = int(state_f['twoJ'])
/tmp/ipykernel_2553/929582128.py:43: SettingWithCopyWarning: 
A value is trying

-------------------------------------------
Excluding transition i = 498
Matched # initial states: 1
Matched # final   states: 0
Matched # Ji             : 1
Matched # Jf             : 7
Matched # Ei             : 1
Matched # Ef             : 0
     Nucleus Elevel(keV)  J  E(keV) dE(keV)  Include  twoJ Element   A  n
4002     36S    4523.0 6  1  4523.0     6.0     True     2       S  36  1
Empty DataFrame
Columns: [Nucleus, Elevel(keV), J, E(keV), dE(keV), Include, twoJ, Element, A, n]
Index: []
A                            36
Element                       S
Z                            16
N                            20
Energy (keV)             4523.0
Energy Unc (keV)              6
Ji                            1
Thalf                0.017 PS 8
E_gamma (keV)            4522.2
E_gamma Unc (keV)            15
M Gamma                      M1
B_exp (W.u.)               0.01
B_exp Unc (W.u.)              5
twoJi                         2
Nucleus                     36S
Jf                 

/tmp/ipykernel_2553/929582128.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Include'].loc[i] = False
/tmp/ipykernel_2553/929582128.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jf'].loc[i] = list(state_f['J'])[0]
/tmp/ipykernel_2553/929582128.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['twoJf'].loc[i] = int(state_f['twoJ'])
/tmp/ipykernel_2553/929582128.py:43: SettingWithCopyWarning: 
A value is trying

In [49]:
# df_levels['Nucleus']==df['Nucleus'].loc[i]

In [50]:
df[df['Include']==False]

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.),twoJi,Nucleus,Jf,twoJf,ni,nf,Include
7,18,F,9,9,1041.55,8,0,1.77 FS 31,1041.00,,M1,10.90000,21,0,18F,None,None,None,None,False
10,18,F,9,9,3061.84,18,2,0.83 FS LT,3061.00,,M1,0.20000,GT,4,18F,None,None,None,None,False
19,18,F,9,9,5297.60,15,4,21 FS 3,1940.00,,M1+E2,0.00037,31,8,18F,None,None,None,None,False
21,18,F,9,9,5603.38,27,1,43.3 EV 16,4561.00,,M1,0.00900,30,2,18F,None,None,None,None,False
22,18,F,9,9,5603.38,27,1,43.3 EV 16,2541.00,,M1,1.10000,2,2,18F,None,None,None,None,False
36,18,F,9,9,6310.50,8,3,0.95 KEV 14,3788.00,,M1,0.00600,18,6,18F,None,None,None,None,False
38,18,F,9,9,6310.50,8,3,0.95 KEV 14,2472.00,,M1,0.02500,9,6,18F,None,None,None,None,False
39,18,F,9,9,6385.50,17,2,0.49 KEV 9,2547.00,,M1,0.18000,9,4,18F,None,None,None,None,False
89,21,Ne,10,11,2866.60,2,9/2,40 FS 4,1120.00,3,M1+E2,0.24000,3,9,21Ne,None,None,None,None,False
93,21,Ne,10,11,4684.53,15,3/2,11 FS 3,4683.94,25,M1,0.00710,20,3,21Ne,None,None,None,None,False


In [51]:
df[(df['Include']==True) & (df['Nucleus']=='22Na')]

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),M Gamma,B_exp (W.u.),B_exp Unc (W.u.),twoJi,Nucleus,Jf,twoJf,ni,nf,Include
115,22,Na,11,11,657.00,14,0,19.6 PS 7,73.90,1,M1,2.77000,10,0,22Na,1,2,1,1,True
116,22,Na,11,11,890.89,20,4,10.0 PS 3,890.87,20,M1+E2,0.00028,5,8,22Na,3,6,1,1,True
117,22,Na,11,11,1528.10,3,5,3.42 PS 24,637.50,20,M1+E2,0.00020,7,10,22Na,4,8,1,1,True
119,22,Na,11,11,2968.60,6,3,32 FS 4,1016.80,5,M1,0.65000,9,6,22Na,2,4,3,1,True
120,22,Na,11,11,3059.40,6,2,23 FS 3,2476.20,3,M1+E2,0.00180,7,4,22Na,1,2,2,1,True
121,22,Na,11,11,3059.40,6,2,23 FS 3,1107.60,5,M1,0.68000,9,4,22Na,2,4,2,1,True
123,22,Na,11,11,4071.40,6,4,0.7 FS LT,2088.20,5,M1,3.00000,>,8,22Na,3,6,2,2,True
124,22,Na,11,11,4319.00,5,1,17 FS 10,3662.00,,M1,0.02600,16,2,22Na,0,0,4,1,True
125,22,Na,11,11,4360.00,4,2,5.0 FS 14,2408.10,3,M1,0.24000,7,4,22Na,2,4,3,1,True
126,22,Na,11,11,4710.00,4,5,37 FS 14,638.60,,M1,1.00000,4,10,22Na,4,8,3,2,True


In [52]:
df_output = df[df['Include']==True]

In [53]:
df_output.to_csv('cleaned_sd_M1_data_v3.csv',index=False)